In [3]:
import os
import pickle
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor
import logging
from functools import lru_cache
import time
import requests
import json

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class Response:
    """Structured response data with metadata"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0

class OllamaClient:
    """Handles communication with Ollama API"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self._check_model()
        
    def _check_model(self):
        """Verify model exists in Ollama"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            available_models = [model['name'] for model in response.json()['models']]
            model_base = self.model_name.split('/')[-1].split(':')[0]
            
            if model_base not in available_models:
                logger.warning(f"Model {model_base} not found. Attempting to pull...")
                self._pull_model()
                
    def _pull_model(self):
        """Pull model from Hugging Face"""
        try:
            requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                timeout=600
            )
            logger.info(f"Successfully pulled model {self.model_name}")
        except Exception as e:
            logger.error(f"Failed to pull model: {e}")
            raise

    def generate(self, prompt: str) -> str:
        """Generate response from Ollama model"""
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json={
                    "model": self.model_name,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": 0.7,
                        "top_p": 0.9,
                        "top_k": 40
                    }
                },
                timeout=30
            )
            response.raise_for_status()
            return response.json()["response"]
        except Exception as e:
            logger.error(f"Ollama API error: {e}")
            return f"Error: Failed to generate response - {str(e)}"
    
class OptimizedAgentSystem:
    def __init__(self, 
                 model_path: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L",
                 pickle_file: str = "responses.pkl",
                 cache_size: int = 1000):
        """Initialize the optimized agent system with caching and parallel processing"""
        self.pickle_file = pickle_file
        self.responses: List[Response] = self._load_responses()
        self.executor = ThreadPoolExecutor(max_workers=4)
        self.ollama_client = OllamaClient(model_path)
        
    def _load_responses(self) -> List[Response]:
        """Load responses with error handling and data validation"""
        try:
            if os.path.exists(self.pickle_file):
                with open(self.pickle_file, "rb") as f:
                    data = pickle.load(f)
                    if isinstance(data[0], str):
                        return [Response(text=r, timestamp=time.time()) for r in data]
                    return data
        except Exception as e:
            logger.error(f"Error loading responses: {e}")
        return []

    @lru_cache(maxsize=100)
    def find_similar_insights(self, query: str, top_k: int = 3) -> List[Tuple[Response, float]]:
        """Optimized similarity search using caching and vector operations"""
        if not self.responses:
            return []
            
        query_words = set(query.lower().split())
        scores = []
        
        for response in self.responses:
            if response.error:
                continue
            response_words = set(response.text.lower().split())
            similarity = len(query_words & response_words) / len(query_words | response_words)
            scores.append((response, similarity))
            
        return sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]

    def _process_response(self, response: str) -> Response:
        """Process and validate individual responses"""
        start_time = time.time()
        is_error = response.startswith("Error:") or not response.strip()
        
        return Response(
            text=response,
            timestamp=start_time,
            error=is_error,
            processing_time=time.time() - start_time
        )

    def run_agent_interaction(self, prompt: str) -> Optional[Response]:
        """Enhanced agent interaction with error handling and metrics"""
        try:
            similar = self.find_similar_insights(prompt)
            insights_text = "\n".join([f"Previous insight: {r[0].text[:100]}..." for r in similar])
            
            augmented_prompt = (
                f"Context from similar interactions:\n{insights_text}\n\n"
                f"Query: {prompt}\n"
                "Provide a detailed, accurate response:"
            )
            
            start_time = time.time()
            response_text = self.ollama_client.generate(augmented_prompt)
            processing_time = time.time() - start_time
            
            response = Response(
                text=response_text,
                timestamp=time.time(),
                error=response_text.startswith("Error:"),
                processing_time=processing_time
            )
            
            self.executor.submit(self._save_response, response)
            
            return response
            
        except Exception as e:
            logger.error(f"Error in agent interaction: {e}")
            return Response(text=f"Error: {str(e)}", timestamp=time.time(), error=True)

    def _save_response(self, response: Response):
        """Asynchronous response saving"""
        try:
            self.responses.append(response)
            with open(self.pickle_file, "wb") as f:
                pickle.dump(self.responses, f)
        except Exception as e:
            logger.error(f"Error saving response: {e}")

    def generate_validation_report(self) -> dict:
        """Enhanced validation with detailed metrics"""
        if not self.responses:
            return {"status": "No responses to analyze"}
            
        valid_responses = [r for r in self.responses if not r.error]
        total = len(self.responses)
        valid_count = len(valid_responses)
        
        if not valid_responses:
            return {
                "total_responses": total,
                "valid_responses": 0,
                "error_rate": 1.0,
                "status": "No valid responses to analyze"
            }
        
        lengths = [len(r.text) for r in valid_responses]
        word_counts = [len(r.text.split()) for r in valid_responses]
        processing_times = [r.processing_time for r in valid_responses]
        
        return {
            "total_responses": total,
            "valid_responses": valid_count,
            "error_rate": (total - valid_count) / total if total else 0,
            "length_analysis": {
                "average": np.mean(lengths),
                "min": min(lengths),
                "max": max(lengths),
                "distribution": np.histogram(lengths, bins=3)[0].tolist()
            },
            "vocabulary_analysis": {
                "unique_words": len(set(" ".join(r.text for r in valid_responses).split())),
                "avg_words_per_response": np.mean(word_counts)
            },
            "performance": {
                "avg_processing_time": np.mean(processing_times),
                "max_processing_time": max(processing_times)
            }
        }

def main():
    """Example usage of the optimized system"""
    system = OptimizedAgentSystem(
        model_path="hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L"
    )
    
    print("Initialized Agent System with Gemmasutra-Mini-2B model")
    print("Available commands:")
    print("- Type your prompt to get a response")
    print("- Type 'stats' to see system statistics")
    print("- Type 'exit' to quit")
    
    while True:
        try:
            prompt = input("\nEnter prompt: ").strip()
            
            if prompt.lower() == 'exit':
                break
            elif prompt.lower() == 'stats':
                report = system.generate_validation_report()
                print("\n=== System Report ===")
                for key, value in report.items():
                    print(f"\n{key.replace('_', ' ').title()}:")
                    if isinstance(value, dict):
                        for k, v in value.items():
                            print(f"  - {k.replace('_', ' ').title()}: {v}")
                    else:
                        print(f"  {value}")
            else:
                print("\nGenerating response...")
                response = system.run_agent_interaction(prompt)
                if response:
                    print(f"\nResponse: {response.text}")
                    if response.error:
                        print("⚠️ Error detected in response")
                    print(f"\nProcessing time: {response.processing_time:.2f}s")
                
        except KeyboardInterrupt:
            print("\nExiting gracefully...")
            break
        except Exception as e:
            print(f"\nError: {e}")

if __name__ == "__main__":
    main()

SyntaxError: expected 'except' or 'finally' block (3806006448.py, line 42)

In [ ]:
import os
import pickle
import numpy as np
from dataclasses import dataclass
from typing import List, Tuple, Optional
from concurrent.futures import ThreadPoolExecutor
import logging
from functools import lru_cache
import time
import requests
import json

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

@dataclass
class Response:
    """Structured response data with metadata"""
    text: str
    timestamp: float
    error: bool = False
    processing_time: float = 0.0

class OllamaClient:
    """Handles communication with Ollama API"""
    def __init__(self, model_name: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L", base_url: str = "http://localhost:11434"):
        self.model_name = model_name
        self.base_url = base_url
        self._check_model()
        
    def _check_model(self):
        """Verify model exists in Ollama"""
        try:
            response = requests.get(f"{self.base_url}/api/tags")
            available_models = [model['name'] for model in response.json()['models']]
            model_base = self.model_name.split('/')[-1].split(':')[0]
            
            if model_base not in available_models:
                logger.warning(f"Model {model_base} not found. Attempting to pull...")
                self._pull_model()
        except Exception as e:
            logger.error(f"Error checking model: {e}")
            raise
                
    def _pull_model(self):
        """Pull model from Hugging Face"""
        try:
            requests.post(
                f"{self.base_url}/api/pull",
                json={"name": self.model_name},
                timeout=600
            )
            logger.info(f"Successfully pulled model {self.model_name}")
        except Exception as e:
            logger.error(f"Failed to pull model: {e}")
            raise

    def generate(self, prompt: str) -> str:
        """Generate response from Ollama model"""
        try:
            response = requests.post(
                f"{self.base_url}/api/generate",
                json={
                    "model": self.model_name,
                    "prompt": prompt,
                    "stream": False,
                    "options": {
                        "temperature": 0.7,
                        "top_p": 0.9,
                        "top_k": 40
                    }
                },
                timeout=30
            )
            response.raise_for_status()
            return response.json()["response"]
        except Exception as e:
            logger.error(f"Ollama API error: {e}")
            return f"Error: Failed to generate response - {str(e)}"
    
class OptimizedAgentSystem:
    def __init__(self, 
                 model_path: str = "hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L",
                 pickle_file: str = "responses.pkl",
                 cache_size: int = 1000):
        """Initialize the optimized agent system with caching and parallel processing"""
        self.pickle_file = pickle_file
        self.responses: List[Response] = self._load_responses()
        self.executor = ThreadPoolExecutor(max_workers=4)
        self.ollama_client = OllamaClient(model_path)
        
    def _load_responses(self) -> List[Response]:
        """Load responses with error handling and data validation"""
        try:
            if os.path.exists(self.pickle_file):
                with open(self.pickle_file, "rb") as f:
                    data = pickle.load(f)
                    if isinstance(data[0], str):
                        return [Response(text=r, timestamp=time.time()) for r in data]
                    return data
        except Exception as e:
            logger.error(f"Error loading responses: {e}")
        return []

    @lru_cache(maxsize=100)
    def find_similar_insights(self, query: str, top_k: int = 3) -> List[Tuple[Response, float]]:
        """Optimized similarity search using caching and vector operations"""
        if not self.responses:
            return []
            
        query_words = set(query.lower().split())
        scores = []
        
        for response in self.responses:
            if response.error:
                continue
            response_words = set(response.text.lower().split())
            similarity = len(query_words & response_words) / len(query_words | response_words)
            scores.append((response, similarity))
            
        return sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]

    def _process_response(self, response: str) -> Response:
        """Process and validate individual responses"""
        start_time = time.time()
        is_error = response.startswith("Error:") or not response.strip()
        
        return Response(
            text=response,
            timestamp=start_time,
            error=is_error,
            processing_time=time.time() - start_time
        )

    def run_agent_interaction(self, prompt: str) -> Optional[Response]:
        """Enhanced agent interaction with error handling and metrics"""
        try:
            similar = self.find_similar_insights(prompt)
            insights_text = "\n".join([f"Previous insight: {r[0].text[:100]}..." for r in similar])
            
            augmented_prompt = (
                f"Context from similar interactions:\n{insights_text}\n\n"
                f"Query: {prompt}\n"
                "Provide a detailed, accurate response:"
            )
            
            start_time = time.time()
            response_text = self.ollama_client.generate(augmented_prompt)
            processing_time = time.time() - start_time
            
            response = Response(
                text=response_text,
                timestamp=time.time(),
                error=response_text.startswith("Error:"),
                processing_time=processing_time
            )
            
            self.executor.submit(self._save_response, response)
            
            return response
            
        except Exception as e:
            logger.error(f"Error in agent interaction: {e}")
            return Response(text=f"Error: {str(e)}", timestamp=time.time(), error=True)

    def _save_response(self, response: Response):
        """Asynchronous response saving"""
        try:
            self.responses.append(response)
            with open(self.pickle_file, "wb") as f:
                pickle.dump(self.responses, f)
        except Exception as e:
            logger.error(f"Error saving response: {e}")

    def generate_validation_report(self) -> dict:
        """Enhanced validation with detailed metrics"""
        if not self.responses:
            return {"status": "No responses to analyze"}
            
        valid_responses = [r for r in self.responses if not r.error]
        total = len(self.responses)
        valid_count = len(valid_responses)
        
        if not valid_responses:
            return {
                "total_responses": total,
                "valid_responses": 0,
                "error_rate": 1.0,
                "status": "No valid responses to analyze"
            }
        
        lengths = [len(r.text) for r in valid_responses]
        word_counts = [len(r.text.split()) for r in valid_responses]
        processing_times = [r.processing_time for r in valid_responses]
        
        return {
            "total_responses": total,
            "valid_responses": valid_count,
            "error_rate": (total - valid_count) / total if total else 0,
            "length_analysis": {
                "average": np.mean(lengths),
                "min": min(lengths),
                "max": max(lengths),
                "distribution": np.histogram(lengths, bins=3)[0].tolist()
            },
            "vocabulary_analysis": {
                "unique_words": len(set(" ".join(r.text for r in valid_responses).split())),
                "avg_words_per_response": np.mean(word_counts)
            },
            "performance": {
                "avg_processing_time": np.mean(processing_times),
                "max_processing_time": max(processing_times)
            }
        }

def main():
    """Example usage of the optimized system"""
    system = OptimizedAgentSystem(
        model_path="hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L"
    )
    
    print("Initialized Agent System with Gemmasutra-Mini-2B model")
    print("Available commands:")
    print("- Type your prompt to get a response")
    print("- Type 'stats' to see system statistics")
    print("- Type 'exit' to quit")
    
    while True:
        try:
            prompt = input("\nEnter prompt: ").strip()
            
            if prompt.lower() == 'exit':
                break
            elif prompt.lower() == 'stats':
                report = system.generate_validation_report()
                print("\n=== System Report ===")
                for key, value in report.items():
                    print(f"\n{key.replace('_', ' ').title()}:")
                    if isinstance(value, dict):
                        for k, v in value.items():
                            print(f"  - {k.replace('_', ' ').title()}: {v}")
                    else:
                        print(f"  {value}")
            else:
                print("\nGenerating response...")
                response = system.run_agent_interaction(prompt)
                if response:
                    print(f"\nResponse: {response.text}")
                    if response.error:
                     print("⚠️ Error detected in response")
                print(f"\nProcessing time: {response.processing_time:.2f}s")
        except KeyboardInterrupt:
                print("\nExiting gracefully...")
                break
        except Exception as e:
                print(f"\nError: {e}")

if __name__ == "__main__":
    main()

2025-02-19 13:52:03,112 - WARNING - Model Gemmasutra-Mini-2B-v1-GGUF not found. Attempting to pull...
2025-02-19 13:52:03,420 - INFO - Successfully pulled model hf.co/TheDrummer/Gemmasutra-Mini-2B-v1-GGUF:Q3_K_L


Initialized Agent System with Gemmasutra-Mini-2B model
Available commands:
- Type your prompt to get a response
- Type 'stats' to see system statistics
- Type 'exit' to quit

Generating response...

Response: To provide an accurate and detailed response to the given query, I would need you to first provide more context or information. Without any specifics, it's not possible for me to give a detailed, accurate answer to this broad question about "how to be good at everything."

If you have specific areas of life that you want advice on improving - like your health, finances, relationships, career etc. - and what you're doing or not doing to improve in those areas right now, then I can provide a more detailed, personalized response tailored specifically to your situation. 

But if you don't have any details about what you're looking for advice on improving, or how you're currently approaching that goal, then I would need you to first tell me what specific area(s) of life you want guidan